In [0]:
import torch
from torch.optim.optimizer import Optimizer
from math import ceil

In [0]:
class GradSliding(Optimizer):
    """Lan's Gradient Sliding algorithm."""
    def __init__(self, params, L, M, D_tilde):
        defaults = dict(L=L, M=M, D_tilde=D_tilde)
        super().__init__(params, defaults)
        self.k = 0
        self.t = 0
        self.mode = 'main'
    
    def upd_main_parameters(self):
        """
        Update parameters of main loop of gradient sliding.
        
        Increment k (counter in main loop). Change mode to PS
        (prox-sliding procedure). Calculate parameters according to the
        formulas in Lan's book:
        gamma, T, beta - formula (8.1.42); T - formula (8.1.42).
        gamma_next is value of gamma in the next iteration.
        """
        self.k += 1
        self.mode = 'PS'

        self.gamma = 3 / (self.k + 2)
        self.gamma_next = 3 / (self.k + 3)
        
        L = self.defaults['L']
        M = self.defaults['M']
        D_tilde = self.defaults['D_tilde']
        T = ceil(M**2 * (self.k + 1)**3 / (D_tilde * L**2))
        self.T = int(T)
        
        self.P = 2 / ((self.T + 1) * (self.T + 2))
        self.beta = 9 * L * (1 - self.P) / (2 * (self.k + 1))
        # print(f">>> gamma={self.gamma:.2f}, T={self.T:.2f}, P={self.P:.2f}, beta={self.beta:.2f}")
    
    def upd_PS_parameters(self):
        """
        Update parameters of PS procedure.
        
        Increment t (counter in PS procedure). Calculate p and theta
        according to formula (8.1.39) in Lan's book. If this is the last
        PS iteration, change mode to main and reset counter.
        """
        self.t += 1
        self.p = self.t / 2
        self.theta = 2 * (self.t + 1) / (self.t * (self.t + 3))
        # print(f"p={self.p:.2f}, theta={self.theta:.2f}")
        
        if self.t % self.T == 0:
            self.t = 0
            self.mode = 'main'

    @torch.no_grad()
    def step(self, closure=None):
        """Perform Gradient Sliding step."""
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()
        
        # Part of main loop before PS (prox-sliding) procedure.
        # In this branch, par is x_underbar in notation of Lan's book.
        if self.mode == 'main':
            self.upd_main_parameters()
            for group in self.param_groups:
                for par in group['params']:
                    if par.grad is None:
                        continue
                    
                    state = self.state[par]
                    # State initialization.
                    if len(state) == 0:
                        state['x'] = par.clone()
                        state['x_bar'] = par.clone()
                    
                    state['df_x'] = par.grad
                    # At the beginning of PS procedure, gradient of h
                    # will be calculated at u0 = x.
                    par.copy_(state['x'])
                
        # PS procedure.
        # In this branch, par is u in notation of Lan's book.
        elif self.mode == 'PS':
            self.upd_PS_parameters()
            for group in self.param_groups:
                for par in group['params']:
                    if par.grad is None:
                        continue
                    
                    state = self.state[par]
                    if self.t == 1:
                        state['u_tilde'] = par.clone()
                    
                    dh_u = par.grad

                    # Formula (1) from our report.
                    numerator = self.beta * (state['x'] + self.p * par) \
                              - state['df_x'] - dh_u
                    par.copy_(numerator / (self.beta * (1 + self.p)))
                    
                    state['u_tilde'] = (1 - self.theta) * state['u_tilde'] \
                                     + self.theta * par
                    
                    if self.t % self.T == 0:
                        # Finish PS procedure.
                        state['x'] = par
                        state['x_tilde'] = state['u_tilde']
                
                        # Part of main loop after PS procedure.
                        state['x_bar'] = (1 - self.gamma) * state['x_bar'] \
                                       + self.gamma * state['x_tilde']
                        # Beginning of main loop of new iteration.
                        # Now par is again x_underbar.
                        x_underbar = (1 - self.gamma_next) * state['x_bar'] \
                                   + self.gamma_next * state['x']
                        par.copy_(x_underbar)
                
        return loss

### 1. Model
Linear layer without bias: $g(x) = Ax$, where $A \in \mathbb{R}^{m \times n}$ is a feature matrix, $x \in \mathbb{R}^n$ is a vector of model's parameters.

### 2. RMSE
\begin{equation*}
    h(x) = \sqrt{\frac{1}{m} \| g(x) - b \|^2} = \frac{1}{\sqrt{m}} \| Ax - b \|,
\end{equation*}
where $b$ is target vector. Let us prove that $h(x)$ is Lipschitz continuous:
\begin{equation*}
    |h(x) - h(y)| = \frac{1}{\sqrt{m}} \bigl\lvert \| Ax - b \| - \| Ay - b \| \bigr\rvert \leq \frac{1}{\sqrt{m}} \| (Ax - b) - (Ay - b) \| = \frac{1}{\sqrt{m}} \| A(x - y) \| \leq \frac{\| A \|}{\sqrt{m}} \| x - y \|.
\end{equation*}
Lipschitz constant is $M_h = \frac{\| A \|}{\sqrt{m}}$. There exist the following theorem:
If convex function $h$ is Lipschitz continuous with parameter $M_h$, then for $M = 2M_h$ the following condition holds:
\begin{equation*}
    h(x) \leq h(y)+\left\langle h^{\prime}(y), x-y\right\rangle+M\|x-y\|\quad \forall x, y
\end{equation*}
In our case, $M = \frac{2}{\sqrt{m}} \| A \|$.

### 3. $L_2$ regularization
\begin{equation*}
    f(x) = \frac{\lambda}{n} \| x \|^2,
\end{equation*}
where $\lambda > 0$ is regularization coefficient. Let us show that $f(x)$ has Lipschitz continuous gradient:
\begin{equation*}
    \nabla f(x) = \frac{2 \lambda}{n} x,\quad \| \nabla f(x) - \nabla f(y) \| = \frac{2 \lambda}{n} \| x - y \|.
\end{equation*}
Thus, $f$ has Lipschitz continuous gradient with parameter $L = \frac{2 \lambda}{n}$.

### 4. Optimization problem
\begin{equation*}
    \min_{x \in \mathbb{R}^n} \left\{ h(x) + f(x) = \frac{1}{\sqrt{m}} \| Ax - b \| + \frac{\lambda}{n} \| x \|^2 \right\}
\end{equation*}
Let us bound squared norm of solution. For $x$ to be at least as good as $0$, the following condition should hold:
\begin{equation*}
    \frac{1}{\sqrt{m}} \| b \| \geq \frac{\lambda}{n} \| x \|^2 \iff \| x \|^2 \leq \frac{n}{\lambda \sqrt{m}} \| b \| =: R^2
\end{equation*}
Now we can reformulate the problem as follows:
\begin{equation*}
    \min_{x \in X} \left\{ h(x) + f(x) \right\},\quad X:= \{ x \in \mathbb{R}^n \bigm| \| x \| \leq R \}
\end{equation*}

### 5. Bregman's distance
Euclidean setup: $V(x, y) = \frac{1}{2} \| x - y \|^2$. Let us find such $D_X^2$ that $V(x, y) \leq D_X^2\ \forall x, y \in X$.
\begin{equation*}
    \frac{1}{2} \| x - y \|^2 \leq \frac{1}{2} \| 2 x_{\text{max}} \|^2 = 2 R^2 = \frac{2n}{\lambda \sqrt{m}} \| b \| =: D_X^2.
\end{equation*}
One of the parameters of gradient sliding is $\tilde{D} := \frac{81}{16} D_X^2 = \frac{81 n}{8 \lambda \sqrt{m}} \| b \|$ (see page 497 of Lan's book, Corollary 8.2).

In [24]:
import numpy as np

m = 100
n = 10
noise_std = 0.01
reg_coef = 1.

np.random.seed(0)
A = np.random.rand(m, n)
np.random.seed(0)
x_true = np.random.rand(n)
np.random.seed(0)
b = A @ x_true + noise_std * np.random.rand(m)

M = 2 * np.linalg.norm(A, ord=2) / np.sqrt(m)
L = 2 * reg_coef / n
D_tilde = 81 * n * np.linalg.norm(b, ord=2) / (8 * reg_coef * np.sqrt(m))
print(f"Algorithm parameters:\nL={L:.2f}, M={M:.2f}, D_tilde={D_tilde:.2f}")

Algorithm parameters:
L=0.20, M=3.21, D_tilde=317.02


In [0]:
X_train = torch.tensor(A, dtype=torch.float)
y_train = torch.tensor(b.reshape(-1, 1), dtype=torch.float)

In [0]:
import torch.nn as nn

model = nn.Linear(10, 1, bias=False)
opt = GradSliding(model.parameters(), L, M, D_tilde)

mse = nn.MSELoss()

In [50]:
for i in range(1000):

    y_pred = model(X_train)
    if opt.mode == 'main':
        # Regularization term.
        reg = 0
        for W in model.parameters():
            reg = reg + W.norm(2)**2
        loss = reg_coef * reg / n
        
        with torch.no_grad():
            sum_loss = torch.sqrt(mse(y_pred, y_train)) + loss
        print(f"k = {opt.k}, loss = {sum_loss:.2f}")
        if i > 0:
            print(f"T = {opt.T}")
    else:
        # RMSE term.
        loss = torch.sqrt(mse(y_pred, y_train))
    opt.zero_grad()
    loss.backward()
    opt.step()
    if i == 0:
        print(f"T = {opt.T}")

k = 0, loss = 3.68
T = 7
k = 1, loss = 1.10
T = 7
k = 2, loss = 0.47
T = 22
k = 3, loss = 0.84
T = 53
k = 4, loss = 0.54
T = 102
k = 5, loss = 0.62
T = 176
k = 6, loss = 0.51
T = 280
